In [1]:
# Version 2

In [1]:
import sys
from pathlib import Path

# 1. Locate project root (one level above src)
project_root = Path().resolve().parent

# 2. Prepend it so that `api` is discoverable
sys.path.insert(0, str(project_root))

In [2]:
import re
from langchain_core.messages import AIMessage, HumanMessage
from api.prompt.templates import template
from api.chain.chains import guardrail_chain, feature_extractor_chain, auditor_chain, fixer_chain
from api.rag.vector_store import format_context
from langchain_core.exceptions import OutputParserException

/home/sudip/PycharmProjects/hackathon-inf/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def regex_extract(sql: str) -> list[dict]:
    tables = re.findall(r'(?:FROM|INTO|TABLE)\s+([A-Za-z0-9_]+)', sql, re.IGNORECASE)
    cols   = re.findall(r'([A-Za-z0-9_]+)\s+(?:NUMBER|VARCHAR2|DATE)', sql, re.IGNORECASE)
    return [
        {"table_name": t, "columns": list(set(cols)), "client_name": ""}
        for t in set(tables)
    ]


In [4]:
def start_chat(question: str, history: list[dict]) -> tuple[str, str | None]:
    # format history for LangChain
    chat_history = []
    for msg in history:
        if not msg['message']:
            msg['message'] = ""
        if msg['role'] == 'user':
            chat_history.append(HumanMessage(content=msg['message']))
        else:
            chat_history.append(AIMessage(content=msg['message']))

    try:
        # Guardrail
        ai_gr = guardrail_chain.invoke({"history": chat_history, "question": question})
        if not ai_gr.get("valid_input"):
            return ai_gr.get("markdown"), None

        # Feature extraction with fallback
        try:
            ai_fe = feature_extractor_chain.invoke({"history": chat_history, "question": question})
        except:
            ai_fe = {}
        features = ai_fe.get("tables", [])
        if not features:
            features = regex_extract(question)
            ai_fe = {
                "tables": features,
                "markdown": "Extracted tables and columns via regex fallback."
            }
        print(features)

        # Build RAG context
        context = ""
        if features:
            kb = format_context(features)
            context = template["context"].invoke({"knowledgebase": kb}).to_string()

        # Auditor
        ai_au = auditor_chain.invoke({
            "history": chat_history,
            "context": context,
            "question": question
        })
        if not ai_au.get("issues_found"):
            summary = f"{ai_fe['markdown']}\n{ai_au['markdown']}"
            return summary, ai_au["markdown"]
        print(ai_au)

        # Fixer
        ai_fx = fixer_chain.invoke({
            "history": chat_history,
            "audit": ai_au,
            "context": context,
            "question": question
        })
        print(ai_fx)

        result = f"{ai_fe['markdown']}\n{ai_au['markdown']}\n{ai_fx['markdown']}"
        return result, ai_fx["markdown"]

    except OutputParserException:
        return "Please provide the PL/SQL script that needs to be reviewed and fixed.", None

In [5]:
# Users input
query = """
Can you please verify this script?

BEGIN
INSERT INTO standard_eligibility (
    member_id, first_name, last_name, date_of_birth, gender, address, city, state, zip_code, plan_id, group_number, coverage_start_date, coverage_end_date, coverage_level_code, subscriber_indicator, product_category_code, plan_type, market_category_code
)
SELECT
    mem_member_id AS member_id,
    first_name_id AS first_name,
    mem_last_name AS last_name,
    date_of_birth AS date_of_birth,
    phm_gender_val AS gender,
    clm_address AS address,
    city AS city,
    rx_state_val AS state,
    phm_zip_code_val AS zip_code,
    mem_plan_id AS plan_id,
    clm_group_number AS group_number,
    clm_coverage_start_date AS coverage_start_date,
    phm_coverage_end_date AS coverage_end_date,
    mem_coverage_level_code AS coverage_level_code,
    clm_subscriber_indicator AS subscriber_indicator,
    rx_product_category_code_val AS product_category_code,
    rx_plan_type AS plan_type,
    rx_market_category_code_val AS market_category_code
FROM eligibility_payer_1;
END;
/

"""

In [6]:
history = []

In [7]:
def ask(question: str):
    """Send question + history to the bot, update history, and return the combined markdown."""
    response, markdown = start_chat(question, history)
    history.append({'role': 'user', 'message': question})
    # if markdown is None, fall back to raw response
    history.append({'role': 'assistant', 'message': markdown or response})
    return response


In [10]:
# Usage:
# answer = ask(query)
answer = ask("Can you please use client UHC for table eligibility_payer_1?")
# answer = ask("Please use the one from our initial conversation.")


[{'table_name': 'standard_eligibility', 'columns': ['member_id', 'first_name', 'last_name', 'date_of_birth', 'gender', 'address', 'city', 'state', 'zip_code', 'plan_id', 'group_number', 'coverage_start_date', 'coverage_end_date', 'coverage_level_code', 'subscriber_indicator', 'product_category_code', 'plan_type', 'market_category_code'], 'client_name': 'Default'}, {'table_name': 'eligibility_payer_1', 'columns': ['mem_member_id', 'first_name_id', 'mem_last_name', 'date_of_birth', 'phm_gender_val', 'clm_address', 'city', 'rx_state_val', 'phm_zip_code_val', 'mem_plan_id', 'clm_group_number', 'clm_coverage_start_date', 'phm_coverage_end_date', 'mem_coverage_level_code', 'clm_subscriber_indicator', 'rx_product_category_code_val', 'rx_plan_type', 'rx_market_category_code_val'], 'client_name': 'United Healthcare'}]
{'issues_found': True, 'original_script': 'BEGIN\nINSERT INTO standard_eligibility (\n    member_id, first_name, last_name, date_of_birth, gender, address, city, state, zip_code, 

In [11]:
print(answer)

The script involves inserting data into the `standard_eligibility` table from the `eligibility_payer_1` table, which is associated with United Healthcare.
### Script Verification

**Original Script:**

```sql
BEGIN
INSERT INTO standard_eligibility (
    member_id, first_name, last_name, date_of_birth, gender, address, city, state, zip_code, plan_id, group_number, coverage_start_date, coverage_end_date, coverage_level_code, subscriber_indicator, product_category_code, plan_type, market_category_code
)
SELECT
    mem_member_id AS member_id,
    first_name_id AS first_name,
    mem_last_name AS last_name,
    date_of_birth AS date_of_birth,
    phm_gender_val AS gender,
    clm_address AS address,
    city AS city,
    rx_state_val AS state,
    phm_zip_code_val AS zip_code,
    mem_plan_id AS plan_id,
    clm_group_number AS group_number,
    clm_coverage_start_date AS coverage_start_date,
    phm_coverage_end_date AS coverage_end_date,
    mem_coverage_level_code AS coverage_level_code,

In [16]:
history

[{'role': 'user',
  'message': '\nCan you please verify this script?\n\nBEGIN\nINSERT INTO standard_eligibility (\n    member_id, first_name, last_name, date_of_birth, gender, address, city, state, zip_code, plan_id, group_number, coverage_start_date, coverage_end_date, coverage_level_code, subscriber_indicator, product_category_code, plan_type, market_category_code\n)\nSELECT\n    mem_member_id AS member_id,\n    first_name_id AS first_name,\n    mem_last_name AS last_name,\n    date_of_birth AS date_of_birth,\n    phm_gender_val AS gender,\n    clm_address AS address,\n    city AS city,\n    rx_state_val AS state,\n    phm_zip_code_val AS zip_code,\n    mem_plan_id AS plan_id,\n    clm_group_number AS group_number,\n    clm_coverage_start_date AS coverage_start_date,\n    phm_coverage_end_date AS coverage_end_date,\n    mem_coverage_level_code AS coverage_level_code,\n    clm_subscriber_indicator AS subscriber_indicator,\n    rx_product_category_code_val AS product_category_code,\n  